<a href="https://colab.research.google.com/github/rim-yu/procam-calibration/blob/master/procam_calibration_201118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# You need to mount your google drive  to the /content/gdrive folder of your virtual computer
# located in the colab server

from google.colab import drive
drive.mount("/content/gdrive")
#drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%xmode Verbose
%pdb on

Exception reporting mode: Verbose
Automatic pdb calling has been turned ON


In [3]:
import os
import os.path
from pathlib import Path
#cf: https://treyhunner.com/2018/12/why-you-should-be-using-pathlib/
import glob
import argparse
import cv2
import numpy as np
import json
from matplotlib import pyplot as plt 
from google.colab.patches import cv2_imshow

In [4]:
def main(proj_height, proj_width, chess_cols, chess_rows, chess_block_size,
    graycode_step, black_thr, white_thr,  cameraParamFile):

    proj_shape = (proj_height, proj_width)
    chess_shape = (chess_cols, chess_rows) # (10, 7)
    chess_block_size = chess_block_size
    gc_step = graycode_step
    black_thr = black_thr
    white_thr = white_thr 
    camera_param_file = cameraParamFile 

    dirnames = sorted(glob.glob('./capture_*'))
    if len(dirnames) == 0: 
        print('Directories \'./capture_*\' were not found')
        return

    print('Searching input files ...')
    used_dirnames = [] 
    gc_fname_lists = [] 
    for dname in dirnames:
        #gc_fnames = sorted(glob.glob(dname + '/graycode_*'))
        gc_fnames = sorted(glob.glob(dname + '/graycode*'))
        if len(gc_fnames) == 0:  
            continue
        used_dirnames.append(dname) 
        gc_fname_lists.append(gc_fnames) 
        print(' \'' + dname + '\' was found')

    camP = None 
    cam_dist = None 
    path, ext = os.path.splitext(camera_param_file) 
    if(ext == ".json"): 
        camP, cam_dist = loadCameraParam(camera_param_file) 
        print('load camera parameters')
        print(camP)
        print(cam_dist)

    calibrate(used_dirnames, gc_fname_lists,
              proj_shape, chess_shape, chess_block_size, gc_step, black_thr, white_thr,
               camP, cam_dist) 

In [5]:
def printNumpyWithIndent(tar, indentchar):
    print(indentchar + str(tar).replace('\n', '\n' + indentchar))  

In [6]:
def loadCameraParam(json_file):
    with open(json_file, 'r') as f:
        param_data = json.load(f)
        P = param_data['camera']['P']
        d = param_data['camera']['distortion']
        return np.array(P).reshape([3,3]), np.array(d) # ok 

In [68]:
def calibrate(dirnames, gc_fname_lists, proj_shape, chess_shape, chess_block_size, gc_step, black_thr, white_thr, camP, camD):

    objps = np.zeros((chess_shape[0]*chess_shape[1], 3), np.float32) 
    # 각 objps 마다 (각 코너 포인트마다) x, y, z 좌표를 저장한다. 저장 장소를 만드는데 모든 값을 0으로 초기화. 10 by 7 array 를 만들고 각 array 값은 좌표임.  
    # 첫 번째 인덱스는 모두 사용, 두 번째 인덱스는 2까지만 사용.
    objps[:, :2] = chess_block_size * \
        np.mgrid[0:chess_shape[0], 0:chess_shape[1]].T.reshape(-1, 2)

    print('Calibrating ...') 
    gc_height = int((proj_shape[0]-1)/gc_step)+1 
    gc_width = int((proj_shape[1]-1)/gc_step)+1 
    # 스트라이프의 가로/세로 갯수 정함. 

    graycode = cv2.structured_light_GrayCodePattern.create(
        gc_width, gc_height) 
    graycode.setBlackThreshold(black_thr)  
    graycode.setWhiteThreshold(white_thr) 


    cam_shape = cv2.imread(gc_fname_lists[0][0], cv2.IMREAD_GRAYSCALE).shape # 카메라 이미지 shape. 
    # cam_shape = (1600, 2400) 
    patch_size_half = int(np.ceil(cam_shape[1] / 180)) # ! 왜 180 으로 나누는지 알아보기. !
    # 2400/180 = 13.33 ... 
    print('  patch size :', patch_size_half * 2 + 1) #

    cam_corners_list = [] 
    cam_objps_list = [] 
    cam_corners_list2 = [] 

    proj_corners_list = [] 
    proj_objps_list = [] 
    
    for dname, gc_filenames in zip(dirnames, gc_fname_lists):
        print('  checking \'' + dname + '\'') 
        if len(gc_filenames) != graycode.getNumberOfPatternImages() + 2: 
          # graycode.getNumberOfPatternImages() = 48
          # gcfiles include black and white imgs 
            print('Error : invalid number of images in \'' + dname + '\'')
            return None 

        graycode_imgs = [] # imgs -> graycode_imgs
        subpixel_imgpoints = [] # _imgpoints -> subpixel_imgpoints

        for fname in gc_filenames: 
            img = cv2.imread(fname)    
            _gray = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)

            if cam_shape != _gray.shape: 
                print('Error : image size of \'' + fname + '\' is mismatch')
                return None 
            graycode_imgs.append(_gray)
        # end of for loop.

        # 맨 뒤 두 장이 black, white img 임
        black_img = graycode_imgs.pop() 
        white_img = graycode_imgs.pop() 
        # white_img is the image where chessboard corners are clearly seen. 

        res, cam_img_corners = cv2.findChessboardCorners(white_img, chess_shape) # cam_corners -> cam_img_corners
        if not res: 
            print('Error : chessboard was not found in \'' +
                  gc_filenames[-2] + '\'')  
            return None

        _ret, _corners = cv2.findChessboardCorners(white_img, chess_shape, None)

        # # draw the found corners
        if _ret == True: # True 면 코너점들을 찾은 것임. 함수가 성공했단 뜻.

        #    cv2.drawChessboardCorners(white_img, chess_shape,  _corners, _ret);
  
        #    # I need the first corner at top-left
        #    # if( centers.front().y > centers.back().y){
        #    #  std::cout << "Reverse order\n";
        #    #  std::reverse(centers.begin(),centers.end());
        #    # }

        #    font = cv2.FONT_HERSHEY_PLAIN
        #    fontScale = 3
        #    color = (0,255,0)
        #    thickness = 2 #px
        #    # chess_shape[1] = the number of rows in the chessboard
        #    # chess_shape[0] = the number of cols in the chessboard
        #    for r in range(0, chess_shape[1]):
        #        for c in range(0, chess_shape[0]):
        #         coord = '(' + str(r) + ',' + str(c) + ')'  
        #         corner_position = _corners[r* chess_shape[0] + c]
        #         origin_x = corner_position[0][0]
        #         origin_y = corner_position[0][1]
        #         origin = (origin_x, origin_y) 
        #         # print("(r, c):", (r, c), origin) 
        #         # cv::putText (InputOutputArray img, const String &text, Point org, int fontFace, double fontScale, Scalar color, int thickness=1, int lineType=LINE_8, bool bottomLeftOrigin=false)
        #         new_white_img_1 = cv2.putText(white_img, coord, origin, font, fontScale, color, thickness, cv2.LINE_AA)
        #        #end of for
        #    #end of for
        #    # Displaying the image
        #    window_name ="Window For CornerPoints (NonSubPixel)"
        #    cv2_imshow(new_white_img_1)
            # window 사이즈 5~11 변경하기.
            # 100 : 오리지널 점이 있고, 윈도우 범위 내에서 새로운 점을 찾음. 반복적으로 찾아가는데 최대 반복 횟수. 여기 걸려서 끝나는 경우가 대부분. ,
            # 0.001 : 새로운 픽셀이 만들어졌는데 이전 픽셀과 다음 픽셀 차이가 0.001 픽셀 이내면 멈춤. -> 굉장히 작게 잡은 것임.              
          criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
          _ret, subPixelCorners = cv2.cornerSubPix(white_img, _corners, (11, 11), (-1, -1), criteria)
          #  # draw the found corners
          #  if _ret == True: # True 면 코너점들을 찾은 것임. 함수가 성공했단 뜻.
          #     cv2.drawChessboardCorners(white_img, chess_shape,  subPixelCorners, _ret);
          #     # I need the first corner at top-left
          #     # if( centers.front().y > centers.back().y){
          #     #  std::cout << "Reverse order\n";
          #     #  std::reverse(centers.begin(),centers.end());
          #     # }

          #     font = cv2.FONT_HERSHEY_PLAIN
          #     fontScale = 3
          #     color = (0,255,0)
          #     thickness = 2 #px

          #     for r in range(0, chess_shape[1]):
          #         for c in range(0, chess_shape[0]):
          #           coord = '(' + str(r) + ',' + str(c) + ')'  
          #           corner_position = subPixelCorners[r* chess_shape[0] + c]
          #           origin_x = corner_position[0][0]
          #           origin_y = corner_position[0][1]
          #           origin = (origin_x, origin_y)
          #           # print(origin)
          #           # print("(r, c):", (r, c), origin) 
          #           new_white_img_2 = cv2.putText(white_img, coord, origin, font, fontScale, color, thickness, cv2.LINE_AA)
          #         #end of for
          #     #end of for
          #     # Displaying the image
          #     window_name ="Window For CornerPoints (SubPixel)"
          #     cv2_imshow(new_white_img_2)              
        else :
            print('Error : chessboard corners were not found in \'' + gc_filenames[-2] + '\'') # 0 이면 첫 번째 -1 이면 맨 끝에 -2 맨 끝에서 두 번째. white img 임. 
            return None

        cam_objps_list.append(objps) 
        # cam_corners_list.append(cam_img_corners) # cam_img_corners 는 subpixel point 가 아님. 
        cam_corners_list.append(subPixelCorners) # _corners 는 sub pixel image point 임. pixel 보다 더 정확한 것임. 

        proj_objps = []
        proj_corners = []
        cam_corners = []

        for (corner, objp) in zip(cam_img_corners, objps): # 코너점에 대응되는 objp 을 구하려함. 
        # cam_img_corners is an array of array of point.
        # corner is an array of point.
        # corner[0] is point. 
        # corner[0][0]는 그 포인트의 x 좌표. corner[0][1]는 그 포인트의 y 좌표. 
        # corner[1] does not exist. corner is actually a point but it is bracketed. 
            c_x = int(round(corner[0][0])) # 코너의 x 좌표.
            c_y = int(round(corner[0][1])) # 코너의 y 좌표. 
            src_points = []
            dst_points = []
            # dx, dy 는 원 블럭 내에 포인트. 그 블럭은 c_x, c_y 를 left-top 코너로 하는 블럭. 
            # patch 라는 건 블럭을 몇 개의 구간으로 나눌 것이냐. 한 블럭의 length 가 96[mm] 임. 13.33 ... 적당히 구간 나누기. 
            # 구간을 많이 나누면 더 좋을 것 같다. 
            for dx in range(-patch_size_half, patch_size_half + 1): # patch_size_half = 13.33 ... 
                for dy in range(-patch_size_half, patch_size_half + 1):
                    x = c_x + dx 
                    y = c_y + dy

                    if int(white_img[y, x]) - int(black_img[y, x]) <= black_thr: 
                        continue

                    err, proj_pix = graycode.getProjPixel(graycode_imgs, x, y) 
                    # x, y 가 image plane 상의 픽셀 좌표. 카메라 이미지 픽셀 좌표에 대응되는 프로젝터 화면 상의 좌표를 구하는 것임.
                    # 그간 구한 graycode image 를 다씀. 

                    if not err:
                        src_points.append((x, y))   
                        dst_points.append(gc_step*np.array(proj_pix))
                    #else:
                    #  print("decoding of projector points failed.")
                    #  return None

            if len(src_points) < patch_size_half**2: 
                print(
                    '    Warning : corner', c_x, c_y,
                    'was skiped because decoded pixels were too few (check your images and threasholds)')
                continue # go to the for loop again.
                # 특정 코너점을 찾으려다가 에러가 남. 카메라 이미지 상의 점과 대응되는 점을 찾을 수가 없다.


            h_mat, inliers = cv2.findHomography(
                np.array(src_points), np.array(dst_points))
            point = h_mat@np.array([corner[0][0], corner[0][1], 1]).transpose()
            point_pix = point[0:2]/point[2]
            proj_objps.append(objp)
            proj_corners.append([point_pix])
            cam_corners.append(corner)
        # end of for loop : for (corner, objp) in zip(cam_img_corners, objps)

        if len(proj_corners) < 3:
            print('Error : too few corners were found in \'' +
                  dname + '\' (less than 3)')
            return None

        proj_objps_list.append(np.float32(proj_objps)) # 한 체스보드 방향에 대해서 proj_objps 생성됨. 
        # 리스트의 리스트. 모든 체스보드 방향에 대해서 체스보드 상의 코너점에 대응되는 프로젝터 데이터를 다 모으는 것임.
        # 그것을 cv2.calibrateCamera 함수에 넘긴다.
        proj_corners_list.append(np.float32(proj_corners))
        cam_corners_list2.append(np.float32(cam_corners))
    # end of for loop : for dname, gc_filenames in zip(dirnames, gc_fname_lists)
    print('Initial solution of camera\'s intrinsic parameters')
    cam_rvecs = []
    cam_tvecs = []
    criteria = (TermCriteria.COUNT+TermCriteria.EPS, 30, DBL_EPSILON) 
    
    if(camP is None): # calibrate camera by using camera corner point list and camera object point list. 
        ret, cam_int, cam_dist, cam_rvecs, cam_tvecs = cv2.calibrateCamera(
            cam_objps_list, cam_corners_list, _gray.shape[::-1], None, None, None, None) 
        print('  RMS :', ret)

    else:
        for objp, corners in zip(cam_objps_list, cam_corners_list):
            ret, cam_rvec, cam_tvec = cv2.solvePnP(objp, corners, camP, camD) 
            cam_rvecs.append(cam_rvec)
            cam_tvecs.append(cam_tvec)
            print('  RMS :', ret)

        cam_int = camP
        cam_dist = camD

    print('  Intrinsic parameters :')
    printNumpyWithIndent(cam_int, '    ')
    print('  Distortion parameters :')
    printNumpyWithIndent(cam_dist, '    ')
    cam_rotation_matrix = np.zeros(shape=(3, 3))
    cv2.Rodrigues(cam_rvecs[0], cam_rotation_matrix) 
    print('  cam_rvecs_capture_00 :')
    printNumpyWithIndent(cam_rotation_matrix, '    ')
    print('  cam_tvecs_capture_00 :')
    printNumpyWithIndent(cam_tvecs[0], '    ')
    print()

    print('Initial solution of projector\'s parameters')
    
    # calibrate projector by using projector corner point list and projector object point list. 

    ret, proj_int, proj_dist, proj_rvecs, proj_tvecs = cv2.calibrateCamera(
        proj_objps_list, proj_corners_list, proj_shape, None, None, None, None)
    print('  RMS :', ret)
    print('  Intrinsic parameters :')
    printNumpyWithIndent(proj_int, '    ')
    print('  Distortion parameters :')
    printNumpyWithIndent(proj_dist, '    ')
    proj_rotation_matrix = np.zeros(shape=(3, 3))
    cv2.Rodrigues(proj_rvecs[0], proj_rotation_matrix) 
    print('  proj_rvecs_capture_00 :')
    printNumpyWithIndent(proj_rotation_matrix, '    ')
    print('  proj_tvecs_capture_00 :')
    printNumpyWithIndent(proj_tvecs[0], '    ') 
    print()

    print('=== Result ===')
    ret, cam_int, cam_dist, proj_int, proj_dist, cam_proj_rmat, cam_proj_tvec, E, F = cv2.stereoCalibrate(
        proj_objps_list, cam_corners_list2, proj_corners_list, cam_int, cam_dist, proj_int, proj_dist, None)
    print('  RMS :', ret)
    print('  Camera intrinsic parameters :')
    printNumpyWithIndent(cam_int, '    ')
    print('  Camera distortion parameters :')
    printNumpyWithIndent(cam_dist, '    ')
    print('  Projector intrinsic parameters :')
    printNumpyWithIndent(proj_int, '    ')
    print('  Projector distortion parameters :')
    printNumpyWithIndent(proj_dist, '    ')
    print('  Rotation matrix / translation vector from camera to projector')
    print('  (they translate points from camera coord to projector coord) :')
    printNumpyWithIndent(cam_proj_rmat, '    ')
    printNumpyWithIndent(cam_proj_tvec, '    ')
    print()

    fovx, fovy, focalLength, principalPoint, aspectRatio = cv2.calibrationMatrixValues(proj_int, (3840, 2160), 16.4, 10.2)
    print("fovx : ", fovx)
    print("fovy : ", fovy)
    print("focalLength : ", focalLength)
    print("principalPoint : ", principalPoint)
    print("aspectRatio : ", aspectRatio)

    fs = cv2.FileStorage('calibration_result.xml', cv2.FILE_STORAGE_WRITE)
    fs.write('img_shape', cam_shape)
    fs.write('rms', ret)
    fs.write('cam_int', cam_int)
    fs.write('cam_dist', cam_dist)
    fs.write('proj_int', proj_int)
    fs.write('proj_dist', proj_dist)
    fs.write('roration', cam_proj_rmat)
    fs.write('translation', cam_proj_tvec)
    fs.release()

In [8]:
path = Path('/content/gdrive/My Drive/danbi-project/data/201110_te/1')
#%cd /content/gdrive/My\ Drive/procam-calibration/sample_data
# !pwd # show me the current working directory

In [9]:
cd /content/gdrive/My Drive/danbi-project/data/201110_te/1

/content/gdrive/My Drive/danbi-project/data/201110_te/1


In [ ]:
ls

-/                      capture_01/  capture_04/  capture_07/  capture_11/
calibration_result.xml  capture_02/  capture_05/  capture_08/  capture_12/
capture_00/             capture_03/  capture_06/  capture_09/


In [69]:
proj_height = 2160 
proj_width = 3840 
chess_cols = 10 
chess_rows = 7  
chess_block_size = 96 
graycode_step = 1
black_thr = 40
white_thr = 5
camera =  ""

main(proj_height, proj_width, chess_cols, chess_rows, chess_block_size,
    graycode_step, black_thr, white_thr,  camera)

Searching input files ...
 './capture_00' was found
 './capture_01' was found
 './capture_02' was found
 './capture_03' was found
 './capture_04' was found
 './capture_05' was found
 './capture_06' was found
 './capture_07' was found
 './capture_08' was found
 './capture_09' was found
 './capture_11' was found
 './capture_12' was found
Calibrating ...
  patch size : 29
  checking './capture_00'


ValueError: ignored

> <ipython-input-68-bdf01fcd57ab>(102)calibrate()
    100             # 0.001 : 새로운 픽셀이 만들어졌는데 이전 픽셀과 다음 픽셀 차이가 0.001 픽셀 이내면 멈춤. -> 굉장히 작게 잡은 것임.
    101           criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
--> 102           _ret, subPixelCorners = cv2.cornerSubPix(white_img, _corners, (11, 11), (-1, -1), criteria)
    103           #  # draw the found corners
    104           #  if _ret == True: # True 면 코너점들을 찾은 것임. 함수가 성공했단 뜻.

--KeyboardInterrupt--
ipdb> q
